In [ ]:
import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
import astropy.constants as c

%matplotlib inline

# GrainPop demo

A `newdust` Grain Population ties together the size distribution, composition, and scattering model physics. There are two types of Grain Populations classes provided:

* `newdust.SingleGrainPop` describes a single grain composition type
* `newdust.GrainPop` is a collection of `SingleGrainPop` objects, and acts much like a dictionary

In [ ]:
import newdust

In [ ]:
# Set up the energy and wavelength grids
EVALS = np.logspace(-1,1,30) * u.keV 
LAMVALS = np.linspace(1000.,8000., 100) * u.angstrom

NH  = 1.e22 # cm^-2
d2g = 0.009 # Dust to gas mass ratio
MD  = NH * c.m_p.to('g').value * d2g # Dust mass column in g cm^-2

print("Dust mass column: {:.1e} g cm^-2".format(MD))

## Shortcuts for constructing a SingleGrainPop object

You can provide the type of size distribution, composition, and scattering model you want by providing a string.

**Size distributions:**
* 'Grain' - a single grain size
* 'Powerlaw'
* 'ExpCutoff'
* Any additional keywords provided to `SingleGrainPop` will be passed to the grain size distribution class

**Compositions:**
* 'Drude' - applies the Drude approximation for the complex index of refraction
* 'Silicate' - uses silicate properties from Draine (2003)
* 'Graphite' - uses perpendicular graphite properties from Draine (2003)

**Scattering Models:**
* 'RG' - uses the Rayleigh Gans approximation (relevant for X-rays)
* 'Mie' - uses the Mie scattering algorithm of Bohren & Hoffman

In [ ]:
# Initialize the grain population
sgpop = newdust.SingleGrainPop('Powerlaw', 'Silicate', 'Mie', md=MD)

# Run the extinction calculation on the grid of wavelengths
sgpop.calculate_ext(LAMVALS)

The built in plotting method, `plot_sdist`, will plot the size distribution using grain size distribution object associated with this SingleGrainPop

In [ ]:
ax = plt.subplot(111)
sgpop.plot_sdist(ax)

The built in plotting method, `plot_ext`, can show all three components of extinction from this population of dust grains: absorption, scattering, and extinction (which is absorption + scattring)

These extinction properties are integrated over the grain size distribution.

This function requres two inputs, a matplotlib axis and a string explaining which properties to plot:
* 'all' will plot scattering, absorption, and extinction
* 'sca' will plot scattering only
* 'abs' will plot absorption only
* 'ext' will plot extinction only

You can change the units on the x-axis with the `unit` keyword. All other keywords are passed to `matplotlib.pyplot.legend` in the case that 'all' extinction propertis are plotted, otherwise keyword arguments are passed to `matplotlib.pyplot.plot`.

In [ ]:
ax = plt.subplot(111)
sgpop.plot_ext(ax, 'all', unit='eV', frameon=False)
plt.semilogy()

Here is an example of using the additional keyword arguments to customize the look of the plot.

In [ ]:
ax = plt.subplot(111)
sgpop.plot_ext(ax, 'ext', color='g', lw=3, label='Extinction')
sgpop.plot_ext(ax, 'sca', color='b', lw=2, label='Scattering')
sgpop.plot_ext(ax, 'abs', color='r', lw=1, label='Absorption')
plt.ylabel(r'$\tau$', size=16)
plt.semilogy()
plt.legend(loc='upper right', frameon=False)

## Customizing a SingleGrainPop

When using the shortcut strings, the composition and scattering models will be initialized with their default parameters. If you need to alter one of the settings, e.g., setting a different composition value for `rho`, then you will need to create the composition object and provide it as input.

Essentially, when initiating a `SingleGrainPop` object, you can provide either a string shortcut or the relevant size distribution, composition, or scattering model.

In the example below, I create a separate `SingleGrainPop` for the perpendicalar and parallel orientations of graphitic grains (see Draine 2003 for details).

In [ ]:
# Here, I import the CmGraphite class into the notebook's top namespace, for ease of reference
from newdust.graindist.composition import CmGraphite

In [ ]:
# Initialize the SingleGrainPop objects
gra_perp = newdust.SingleGrainPop('Powerlaw', CmGraphite(orient='perp'), 'Mie')
gra_para = newdust.SingleGrainPop('Powerlaw', CmGraphite(orient='para'), 'Mie')

# Calculate the extinction over optical wavelengths
gra_perp.calculate_ext(LAMVALS)
gra_para.calculate_ext(LAMVALS)

In [ ]:
# Plot the results
ax = plt.subplot(111)
gra_perp.plot_ext(ax, 'ext', unit='nm', color='k', ls='-', label='Perpendicular')
gra_para.plot_ext(ax, 'ext', unit='nm', color='k', ls='--', label='Parallel')
plt.legend()

# SingleGrainPop objects can be combined into GrainPop

A `GrainPop` object allows us to combine multiple grain size or composition types and run extinction calculations on all of them in one line of code.

A `GrainPop` acts similar to a dictionary when you want to investigate properties of the `SingleGrainPop` collected in this object.

In [ ]:
# Here, I define two grain populations with the same 
# power law size distribution but different compositions
silpop = newdust.SingleGrainPop('Powerlaw', 'Silicate', 'Mie')
grapop = newdust.SingleGrainPop('Powerlaw', 'Graphite', 'Mie')

# Then I put them together into a GrainPop object
# I provide strings as keys to reference each one
# These act like Python dictionary keys
myPop  = newdust.GrainPop([silpop, grapop], keys=['sil','gra'])

In [ ]:
# Now I can do the calculation in one go
myPop.calculate_ext(LAMVALS)

In [ ]:
# Here, I plot the extinction properties of each SingleGrainPop
# Note that I reference the SingleGrainPop using the keys I assigned earlier
ax = plt.subplot(111)

# Plot the extinction properties of the silicate grain population
myPop['sil'].plot_ext(ax, 'ext', color='g', label='Silicate Extinction')

# Plot the extinction properties of the graphitic grain population
myPop['gra'].plot_ext(ax, 'ext', color='b', label='Graphite Extinction')

# And I can plot the total GrainPop extinction properties in the same way as a SingleGrainPop
# This shows the sum of the silicate and graphitic grain extinction values
myPop.plot_ext(ax, 'ext', color='k', lw=2, label='Total')

ax.legend(loc='upper right', frameon=False)
plt.semilogy()

The `GrainPop.info()` method prints several key characteristics of the `SingleGrainPop` objects contained within it. 

In [ ]:
myPop.info()

## Shortcut (helper) functions for common dust models

The `newdust.grainpop` module contains helper functions that provide a shortcut for initiating some common grain size distributions and physics.

### make_MRN

A combination of silicate and graphite grains, following a power law size distribution, with a maximum grain size of 0.3 micron (Mathis, Rumpl, and Nordsieck 1977). This follows the Draine recommendation that the graphitic grains populations is 1/3 (2/3) parallel (perpendicular) and by default follows the Corrales et al. (2016) recommendation that 60% of the grains, by mass, are silicate. You can change the dust mass fraction in silicate grains using the `fsil` keyword.

In [ ]:
mrn = newdust.make_MRN(md=MD)
mrn.calculate_ext(LAMVALS)

In [ ]:
# Plot the total extinction
ax = plt.subplot(111)
mrn.plot_ext(ax, 'all', frameon=False)

In [ ]:
# Show the extinction from each component. N
# Note that they are indexed in the same way as a dictionary.
# You can access the GrainPop keys using the .keys attribut

ax = plt.subplot(111)
mrn.plot_ext(ax, 'ext', color='k', lw=2, label='total')

print("GrainPop keys:", mrn.keys)

for k in mrn.keys:
    mrn[k].plot_ext(ax, 'ext', ls='--', label=k)

ax.legend(loc='upper right', frameon=False)

In [ ]:
mrn.info()

### make_MRN_RGDrude

The Drude approximation describes the complex index of refraction as if the solid is a mass of free electrons, so it is relatively insensitive to compound type. As a consequence, this function returns a `SingleGrainPop` object with the `CmDrude` composition.

The Rayleigh-Gans scattering approximation is most relevant for X-ray wavelengths.

In [ ]:
mrn_rgd = newdust.make_MRN_RGDrude(md=MD)
mrn_rgd.calculate_ext(EVALS)

In [ ]:
ax = plt.subplot(111)
mrn_rgd.plot_ext(ax, 'all')
plt.loglog()

The plot above demonstrates that the Rayleigh-Gans plus Drude approximation produces an extinction model that decays smoothly with $E^{-2}$. There is no absorption component to the RG-Drude model.

In [ ]:
mrn_rgd.info()